In [57]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
import cv2, hashlib
from tqdm import tqdm


INP_DIR = "data/validation"

TRAIN_DIR = "data/validation_processed/training"
TEST_DIR  = "data/validation_processed/testing"
PROC_DIR  = "data/validation_processed/unsorted"

SEED = 136
TRAIN_TEST_SPLIT = 0.2
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

In [58]:
target_dirs = [TRAIN_DIR, TEST_DIR, PROC_DIR]
sub_dirs = ["0_invalid", "1_tec_results"]

print("Creating directory structure...")
for main_dir in target_dirs:
    for sub in sub_dirs:
        os.makedirs(os.path.join(main_dir, sub), exist_ok=True)
print("Done.")

Creating directory structure...
Done.


In [59]:
def bw_image(path_in:str, class_dir:str, name:str) -> None:
    try:
        img = Image.open(path_in)
        bw_img = img.convert("L") 
        bw_img.save(f"{PROC_DIR}/{class_dir}/bw_{name}")
    except Exception as e:
        print(f"Error processing {name}: {e}")

In [60]:
print("Processing images to grayscale...")
count = 0

for class_dir in os.listdir(INP_DIR):
    path = f"{INP_DIR}/{class_dir}"
    if not os.path.isdir(path): continue 
    
    for file in os.listdir(path):
        bw_image(f"{path}/{file}", class_dir, file)
        count += 1

print(f"Processed {count} images.")

Processing images to grayscale...
Processed 822 images.


In [61]:
print("Indexing and splitting data...")
files = []
labels = []

for class_dir in sub_dirs:
    class_path = os.path.join(PROC_DIR, class_dir)
    img_files = glob.glob(f"{class_path}/*")
    
    label = 1 if "1_tec_results" in class_dir else 0
    
    files.extend(img_files)
    labels.extend([label] * len(img_files))

train_val_files, test_files, _, _ = train_test_split(
    files, labels, test_size=TRAIN_TEST_SPLIT, random_state=SEED, stratify=labels
)

print(f"Copying {len(train_val_files)} training files...")
for file_path in train_val_files:
    new_path = file_path.replace("unsorted", "training") 
    shutil.copy(file_path, new_path)

print(f"Copying {len(test_files)} testing files...")
for file_path in test_files:
    new_path = file_path.replace("unsorted", "testing")
    shutil.copy(file_path, new_path)

print("Data split complete.")

Indexing and splitting data...
Copying 657 training files...
Copying 165 testing files...
Data split complete.


In [62]:
print("Loading datasets from disk...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale', # Forces (100, 100, 1) shape
    label_mode="binary"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    label_mode="binary"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    label_mode="binary"
)

print(f"Classes found: {train_ds.class_names}")

Loading datasets from disk...
Found 657 files belonging to 2 classes.
Using 526 files for training.
Found 657 files belonging to 2 classes.
Using 131 files for validation.
Found 165 files belonging to 2 classes.
Classes found: ['0_invalid', '1_tec_results']


In [63]:
def build_micro_cnn(input_shape=(150, 150, 1)):
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=input_shape),

        layers.RandomContrast(0.1),
        layers.RandomTranslation(height_factor=0.05, width_factor=0.05),

        # Convolutional Blocks
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Classification Head
        layers.Flatten(),
        layers.Dense(64, activation='relu'),

        layers.Dropout(0.5),

        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_micro_cnn()
model.summary()

/home/johan/tetris/lib/python3.12/site-packages/keras/src/layers/preprocessing/data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 150, 150, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_4               │ (None, 150, 150, 1)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation_4            │ (None, 150, 150, 1)    │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 148, 148, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 34, 34, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │     1,183,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,239,617 (4.73 MB)

 Trainable params: 1,239,617 (4.73 MB)

 Non-trainable params: 0 (0.00 B)

In [64]:
print("Starting training...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    class_weight={0: 1.0, 1: 5.0}
)
print("Training finished!")

Starting training...
Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.3745 - loss: 1.3667 - val_accuracy: 0.5496 - val_loss: 0.7181
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4981 - loss: 1.2176 - val_accuracy: 0.6794 - val_loss: 0.8455
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.6711 - loss: 0.9534 - val_accuracy: 0.7481 - val_loss: 0.5352
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8137 - loss: 0.6752 - val_accuracy: 0.9466 - val_loss: 0.2173
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.8783 - loss: 0.5104 - val_accuracy: 0.8855 - val_loss: 0.3110
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9068 - loss: 0.3855 - val_accuracy: 0.9542 - val_loss: 0.0956
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.9278 - loss: 0.2920 - val_accuracy: 0.9618 - val_loss: 0.0699
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.9430 - loss: 0.28

In [65]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc*100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.9818 - loss: 0.0891
Test Accuracy: 98.18%


In [56]:
def reset_folders():
    folders_to_clean = [TRAIN_DIR, TEST_DIR, PROC_DIR]
    for folder in folders_to_clean:
        if os.path.exists(folder):
            shutil.rmtree(folder)
            print(f"Cleaned {folder}")
    
    for main_dir in folders_to_clean:
        for sub in ["0_invalid", "1_tec_results"]:
            os.makedirs(os.path.join(main_dir, sub), exist_ok=True)
    print("Folders are ready for a fresh run!")

reset_folders()

Cleaned data/validation_processed/training
Cleaned data/validation_processed/testing
Cleaned data/validation_processed/unsorted
Folders are ready for a fresh run!


### Video Watching Script

In [66]:
INPUT_FOLDER:str = "data/videos"
OUTPUT_FOLDER:str = "data/found_matches"
CONFIDENCE_THRESHOLD:float = 0.92
SKIP_SECONDS:float = 2
COOLDOWN_SECONDS:int = 90

In [67]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

videos = []
videos.extend(glob.glob(os.path.join(INPUT_FOLDER, "*.mp4")))

def preprocess_frame(frame):
    """Matches the training preprocessing: 150x150, Grayscale (0-255)"""
    # Resize to img input size
    small = cv2.resize(frame, (150, 150))
    gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
    
    # Add Batch & Channel Dimensions
    input_tensor = np.expand_dims(gray, axis=0)
    input_tensor = np.expand_dims(input_tensor, axis=-1)

    return input_tensor

for VIDEO_PATH in videos:
    print(f"Scanning video: {VIDEO_PATH}")
    cap = cv2.VideoCapture(VIDEO_PATH)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    COOLDOWN_FRAMES:int = int(COOLDOWN_SECONDS*fps)
    SKIP_FRAMES:int = int(SKIP_SECONDS*fps)

    current_frame = 0
    rounds_found = 0
    cooldown_timer = 0

    print(f"Video FPS: {fps} | Total Frames: {total_frames}")

    pbar = tqdm(total=total_frames, desc=f"Scanning {VIDEO_PATH}", unit="frames")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # End of video
        
        if cooldown_timer > 0:
            cooldown_timer -= 1
        elif current_frame % SKIP_FRAMES == 0:
            
            input_tensor = preprocess_frame(frame)
            
            prediction = model.predict(input_tensor, verbose=0)[0][0]
            
            if prediction > CONFIDENCE_THRESHOLD:
                # Results screen found
                timestamp_sec = int(current_frame / fps)
                print(f"[{timestamp_sec}s] Round Found! (Conf: {prediction:.2f})")

                raw_id = f"{os.path.basename(VIDEO_PATH)}_{timestamp_sec}"
                unique_hash = hashlib.sha256(raw_id.encode()).hexdigest()[:8]
                
                filename = f"match_{rounds_found:03d}_{timestamp_sec}s_{unique_hash}.jpg"
                cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), frame)
                
                rounds_found += 1
                pbar.set_postfix(found=rounds_found)
                
                cooldown_timer = COOLDOWN_FRAMES

        current_frame += 1
        pbar.update(1)

    pbar.close()
    cap.release()
    print(f"Done! Found {rounds_found} rounds.")

Scanning video: data/videos/(Ao Vivo - 21⧸12⧸25) Jogando Tetris Effect： Connected (Modo Competitivo)!!.mp4
Video FPS: 29.99999883666826 | Total Frames: 257880


Scanning data/videos/(Ao Vivo - 21⧸12⧸25) Jogando Tetris Effect： Connected (Modo Competitivo)!!.mp4:  14%|█▎        | 35060/257880 [01:31<08:23, 442.44frames/s, found=1]

[1166s] Round Found! (Conf: 1.00)


Scanning data/videos/(Ao Vivo - 21⧸12⧸25) Jogando Tetris Effect： Connected (Modo Competitivo)!!.mp4:  23%|██▎       | 58897/257880 [02:30<07:28, 443.88frames/s, found=2]

[1960s] Round Found! (Conf: 1.00)


...

[16699s] Round Found! (Conf: 1.00)


Scanning data/videos/I don't know who I am.mp4:  59%|█████▉    | 1008807/1701653 [1:02:04<54:24, 212.26frames/s, found=74]  

[16826s] Round Found! (Conf: 1.00)


Scanning data/videos/I don't know who I am.mp4:  60%|█████▉    | 1017257/1701653 [1:02:37<59:44, 190.94frames/s, found=74]  

KeyboardInterrupt: 

In [68]:
pbar.close()
cap.release()

Scanning data/videos/I don't know who I am.mp4:  60%|█████▉    | 1017275/1701653 [1:02:43<42:11, 270.30frames/s, found=74]
